<h1>Example of use with PyTorch with Convolutional Network by Bresson & Laurent</h1>

Used by Kool et al. (2021) and Joshi et al. (2019) to form routes, using the incidence matrix image as the classes available.

In [1]:
import os

data_files_list = ["./export/"+f for f in os.listdir("./export") ]
instance_dict = {}
for dir_str in data_files_list:
    with open(dir_str, 'r') as text_file:
        cnt = 0
        instance = ""
        for line in text_file:
            if cnt < 9:
                if cnt == 0:
                    instance = line.split()[0]
                    instance_dict[instance] = []
                cnt += 1
                continue
            split_line = line.split()
            instance_dict[instance].append([int(i) for i in split_line])
        text_file.close()

ng_dict = {}
cnt = -1
with open("ng_outs.csv", 'r') as text_file:
    for line in text_file:
        if cnt < 2:
            cnt += 1
            continue
        raw_line = line.strip()
        split_line_list = raw_line.split(sep=";")
        instance = split_line_list[3]
        if instance not in ng_dict:
            ng_dict[instance] = [[0 for i in range(101)]]
        ng_dict[instance].append([0] + [int(i) for i in split_line_list[5:-1]])
        if len(split_line_list[5:-1]) != 100:
            print("case found for instance "+instance)
    text_file.close()

In [2]:
from math import sqrt
import torch
import torch_geometric as tg
from torch_geometric.data import Data
import networkx as nx
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GraphConv, global_add_pool
from torch_geometric.loader import DataLoader

In [3]:
complete_graph_list = []
for i in range(101):
    for j in range(101):
        if i != j:
            complete_graph_list.append([i,j])
edge_index = torch.tensor(complete_graph_list, dtype=torch.long).t().contiguous()
n_edges = len(complete_graph_list)

In [4]:
for instance_name in ng_dict:
    for i in range(101):
        for j in range(101):
            if i == j:
                ng_dict[instance_name][i][j] = 0

In [5]:
data_list = []
for instance_name in ng_dict:
    y = torch.tensor(ng_dict[instance_name], dtype=torch.double)
    x = torch.tensor(instance_dict[instance_name], dtype=torch.double)
    attr = [[i] for i in range(n_edges)]
    loc_dict = {(i[0],j[0]): sqrt((i[1]-j[1])**2 + (i[2]-j[2])**2) for i in instance_dict[instance_name] for j in instance_dict[instance_name]}
    cnt = -1
    for i in range(101):
        for j in range(101):
            if i != j:
                cnt += 1
                attr[cnt].append(loc_dict[i,j])
    attr = torch.tensor(attr, dtype=torch.double)
    pos = []
    for i in instance_dict[instance_name]:
        pos.append([i[1], i[2]])
    pos = torch.tensor(pos, dtype=torch.double)
    data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

In [6]:
# Just to produce edges as an adjacency matrix
complete_adj_matrix_list = [[0 for i in range(101)] for i in range(101)]
for edge in complete_graph_list:
    i, j = edge
    complete_adj_matrix_list[i][j] = 1

In [7]:

class Instances:
    def __init__(self, data_list):
        self.data_list = data_list
    
    def to_torch_geometric(self, start=0, end=-1, batch_size=1):
        return DataLoader(data_list[start:end], batch_size=batch_size)
    
    def to_conv_nets(self, start=0, end=-1, batch_size=1):
        final_data = []
        nodes = []
        nodes_coor = []
        nodes_timew = []
        x_edges = []
        x_edges_values = []
        y_edges = []
        cnt = 0
        current_batch = 0
        for graph in self.data_list[start:end]:
            if cnt >= batch_size:
                cnt = 0
                current_batch += 1
                nodes = torch.tensor(nodes, dtype=torch.long)
                nodes_coor = torch.tensor(nodes_coor, dtype=torch.float)
                nodes_timew = torch.tensor(nodes_timew, dtype=torch.long)
                x_edges = torch.tensor(x_edges, dtype=torch.long)
                x_edges_values = torch.tensor(x_edges_values, dtype=torch.float)
                y_edges = torch.tensor(y_edges, dtype=torch.long)
                final_data.append((x_edges, x_edges_values, nodes, nodes_coor, nodes_timew, y_edges))
                nodes = []
                nodes_coor = []
                nodes_timew = []
                x_edges = []
                x_edges_values = []
                y_edges = []
            nodes.append([i for i in range(101)]) 
            nodes_coor.append(graph.pos.tolist())
            tw = []
            x_raw = graph.x.tolist()
            ## filtering by TW, strict
            complete_adj_matrix_list = [[0 for i in range(101)] for i in range(101)]
            for edge in complete_graph_list:
                i, j = edge
                if x_raw[i][5] + x_raw[i][6] <  x_raw[j][5]:
                    complete_adj_matrix_list[i][j] = 1
            ## end filtering
            for i in range(101):
                tw.append([x_raw[i][4], x_raw[i][5]])
            nodes_timew.append(tw)
            x_edges.append(complete_adj_matrix_list)
            dist_matrix = [[0 for _ in range(101)] for _ in range(101)]
            dist_list = [i for _, i in graph.edge_attr.tolist()]
            pos_dist = 0
            for i in range(101):
                for j in range(101):
                    if i != j:
                        dist_matrix[i][j] = dist_list[pos_dist]
                        pos_dist += 1
            x_edges_values.append(dist_matrix)
            y_edges.append(graph.y.tolist()) #TODO: remove the transpose and also the contiguous when generating y
            cnt += 1
        return final_data


In [16]:
data_source = Instances(data_list)
# dataloader = data_source.to_torch_geometric(start=428, end=458, batch_size=1)
# data_test = data_source.to_torch_geometric(start=458, end=468, batch_size=1)
datatorch = data_source.to_conv_nets(start=428, end=1458, batch_size=1)
torchtest = data_source.to_conv_nets(start=1458, end=1468, batch_size=1)

In [9]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_scatter import scatter
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [10]:
class BatchNormNode(nn.Module):
    """Batch normalization for node features.
    """

    def __init__(self, hidden_dim):
        super(BatchNormNode, self).__init__()
        self.batch_norm = nn.BatchNorm1d(hidden_dim, track_running_stats=False)

    def forward(self, x):
        """
        Args:
            x: Node features (batch_size, num_nodes, hidden_dim)

        Returns:
            x_bn: Node features after batch normalization (batch_size, num_nodes, hidden_dim)
        """
        # The batch norm normalizes the hidden dim over batch and node dimensions
        if x.dim() == 2:
            # If we have sparse version we have only one batch dimension
            # simply perform batch norm over this (so this normalizes over batch and node dimension)
            return self.batch_norm(x)
        x_trans = x.transpose(1, 2).contiguous()  # Reshape input: (batch_size, hidden_dim, num_nodes)
        x_trans_bn = self.batch_norm(x_trans)
        x_bn = x_trans_bn.transpose(1, 2).contiguous()  # Reshape to original shape
        # x_bn2 = self.batch_norm(x.view(-1, x.size(-1))).view_as(x)
        # assert torch.allclose(x_bn, x_bn2, atol=1e-5)
        return x_bn


class BatchNormEdge(nn.Module):
    """Batch normalization for edge features.
    """

    def __init__(self, hidden_dim):
        super(BatchNormEdge, self).__init__()
        self.batch_norm = nn.BatchNorm2d(hidden_dim, track_running_stats=False)

    def forward(self, e):
        """
        Args:
            e: Edge features (batch_size, num_nodes, num_nodes, hidden_dim)

        Returns:
            e_bn: Edge features after batch normalization (batch_size, num_nodes, num_nodes, hidden_dim)
        """
        # The batch norm normalizes the hidden dim over batch and edge dimensions
        if e.dim() == 2:
            # If we have sparse version we have only one batch dimension
            # simply perform batch norm over this (so this normalizes over batch and node dimension)
            # We can use the BatchNorm2d module by inserting dummy dimensions
            return self.batch_norm(e[:, :, None, None]).view_as(e)
        e_trans = e.transpose(1, 3).contiguous()  # Reshape input: (batch_size, hidden_dim, num_nodes, num_nodes)
        e_trans_bn = self.batch_norm(e_trans)
        e_bn = e_trans_bn.transpose(1, 3).contiguous()  # Reshape to original
        return e_bn

class NodeFeatures(nn.Module):
    """Convnet features for nodes.
    
    Using `sum` aggregation:
        x_i = U*x_i +  sum_j [ gate_ij * (V*x_j) ]
    
    Using `mean` aggregation:
        x_i = U*x_i + ( sum_j [ gate_ij * (V*x_j) ] / sum_j [ gate_ij] )
    """
    
    def __init__(self, hidden_dim, aggregation="mean"):
        super(NodeFeatures, self).__init__() # We must always sum, since mean means 'weighted mean' so sum weighted messages
        self.aggregation = aggregation
        self.U = nn.Linear(hidden_dim, hidden_dim, True)
        self.V = nn.Linear(hidden_dim, hidden_dim, True)

    def forward(self, x, edge_gate, edge_index=None):
        """
        Args:
            x: Node features (batch_size, num_nodes, hidden_dim)
            edge_gate: Edge gate values (batch_size, num_nodes, num_nodes, hidden_dim)

        Returns:
            x_new: Convolved node features (batch_size, num_nodes, hidden_dim)
        """

        Ux = self.U(x)  # B x V x H
        Vx = self.V(x)  # B x V x H

        if edge_index is not None:
            # Sparse version
            return self.propagate(edge_index, Ux=Ux, Vx=Vx, edge_gate=edge_gate)

        from torch.utils.checkpoint import checkpoint

        # The rest is a relatively cheap operation that uses a lot of memory
        # No it does not use a lot of memory
        use_checkpoint = False
        if use_checkpoint:
            x_new = checkpoint(self._inner, edge_gate, Ux, Vx)
        else:

            x_new = self._inner(edge_gate, Ux, Vx)
            # print("Dense x", edge_gate.size(), Ux.size(), Vx.size())
            # print(x_new.flatten()[-10:])
        return x_new

    def _inner(self, edge_gate, Ux, Vx):
        use_einsum = False
        use_matmul = False

        if use_einsum:  # Seems to use more memory
            x_add = torch.einsum('bijd,bjd->bid', edge_gate, Vx)
        elif use_matmul:  # Seems to use same memory as einsum, not much faster
            x_add = torch.matmul(
                edge_gate.unsqueeze(1).transpose(1, 4).squeeze(-1),
                Vx.unsqueeze(1).transpose(1, 3)
            ).transpose(1, 3).squeeze(1)
        else:
            Vx = Vx.unsqueeze(1)  # extend Vx from "B x V x H" to "B x 1 x V x H"
            gateVx = edge_gate * Vx  # B x V x V x H
            x_add = torch.sum(gateVx, dim=-2)
        if self.aggregation=="mean":
            x_new = Ux + x_add / (1e-20 + torch.sum(edge_gate, dim=-2))  # B x V x H
        elif self.aggregation=="sum":
            x_new = Ux + x_add  # B x V x H
        return x_new

    def message(self, edge_gate, Vx_j):
        return edge_gate * Vx_j

    def update(self, agg, Ux, edge_gate, edge_index):
        src, tgt = edge_index
        # Aggregate here exactly as in _inner. Normalizing here is more efficient than normalizing the messages.
        if self.aggregation == "mean":
            gate_sum = scatter(edge_gate, tgt, dim=0, dim_size=Ux.size(0), reduce='sum')
            return Ux + agg / (1e-20 + gate_sum)
        assert self.aggregation == "sum"
        return Ux + agg  # Skip connection


class EdgeFeatures(nn.Module):
    """Convnet features for edges.

    e_ij = U*e_ij + V*(x_i + x_j)
    """

    def __init__(self, hidden_dim, directed=False):
        super(EdgeFeatures, self).__init__()
        self.U = nn.Linear(hidden_dim, hidden_dim, True)
        self.V = nn.Linear(hidden_dim, hidden_dim, True)
        self.W = nn.Linear(hidden_dim, hidden_dim, True) if directed else None
        
    def forward(self, x, e, edge_index=None):
        """
        Args:
            x: Node features (batch_size, num_nodes, hidden_dim)
            e: Edge features (batch_size, num_nodes, num_nodes, hidden_dim)

        Returns:
            e_new: Convolved edge features (batch_size, num_nodes, num_nodes, hidden_dim)
        """
        Ue = self.U(e)
        Vx = self.V(x)
        Wx = Vx if self.W is None else self.W(x)  # If self.W is none, graph is undirected
        if edge_index is not None:
            # Sparse version
            src, dst = edge_index
            Wx = Wx[dst]  # = to
            Vx = Vx[src]  # = from
        else:
            Wx = Wx.unsqueeze(1)  # Extend Wx from "B x V x H" to "B x 1 x V x H" = to
            Vx = Vx.unsqueeze(2)  # extend Vx from "B x V x H" to "B x V x 1 x H" = from

        e_new = Ue + Vx + Wx
        return e_new

In [11]:
class ResidualGatedGCNLayer(nn.Module):
    """Convnet layer with gating and residual connection.
    """

    def __init__(self, hidden_dim, aggregation="sum", directed=False):
        super(ResidualGatedGCNLayer, self).__init__()
        self.node_feat = NodeFeatures(hidden_dim, aggregation)
        self.edge_feat = EdgeFeatures(hidden_dim, directed)
        self.bn_node = BatchNormNode(hidden_dim)
        self.bn_edge = BatchNormEdge(hidden_dim)

    def forward(self, x, e, edge_index=None):
        """
        Args:
            x: Node features (batch_size, num_nodes, hidden_dim)
            e: Edge features (batch_size, num_nodes, num_nodes, hidden_dim)

        Returns:
            x_new: Convolved node features (batch_size, num_nodes, hidden_dim)
            e_new: Convolved edge features (batch_size, num_nodes, num_nodes, hidden_dim)
        """
        e_in = e
        x_in = x
        # Edge convolution
        e_tmp = self.edge_feat(x_in, e_in, edge_index)  # B x V x V x H
        # Compute edge gates
        edge_gate = F.sigmoid(e_tmp)
        # Node convolution
        x_tmp = self.node_feat(x_in, edge_gate, edge_index)
        # Batch normalization
        e_tmp = self.bn_edge(e_tmp)
        x_tmp = self.bn_node(x_tmp)
        # ReLU Activation
        e = F.relu(e_tmp)
        x = F.relu(x_tmp)
        # Residual connection
        x_new = x_in + x
        e_new = e_in + e
        return x_new, e_new


class MLP(nn.Module):
    """Multi-layer Perceptron for output prediction.
    """

    def __init__(self, hidden_dim, output_dim, L=2):
        super(MLP, self).__init__()
        self.L = L
        U = []
        for layer in range(self.L - 1):
            U.append(nn.Linear(hidden_dim, hidden_dim, True))
        self.U = nn.ModuleList(U)
        self.V = nn.Linear(hidden_dim, output_dim, True)

    def forward(self, x):
        """
        Args:
            x: Input features (batch_size, hidden_dim)

        Returns:
            y: Output predictions (batch_size, output_dim)
        """
        Ux = x
        for U_i in self.U:
            Ux = U_i(Ux)  # B x H
            Ux = F.relu(Ux)  # B x H
        y = self.V(Ux)  # B x O
        return y

In [12]:
class ResidualGatedGCNModelVRP(nn.Module):
    """Residual Gated GCN Model for outputting predictions as edge adjacency matrices.

    References:
        Paper: https://arxiv.org/pdf/1711.07553v2.pdf
        Code: https://github.com/xbresson/spatial_graph_convnets
    """

    def __init__(self):
        super(ResidualGatedGCNModelVRP, self).__init__()
        self.dtypeFloat = torch.FloatTensor
        self.dtypeLong = torch.LongTensor
        # Define net parameters
        # self.num_nodes = config.num_nodes
        # self.node_dim = config.node_dim
        # self.voc_nodes_in = config['voc_nodes_in']
        # self.voc_nodes_out = config['num_nodes']  # config['voc_nodes_out']
        # self.voc_edges_in = config['voc_edges_in']
        # self.voc_edges_out = config['voc_edges_out']
        # self.hidden_dim = config['hidden_dim']
        # self.num_layers = config['num_layers']
        # self.mlp_layers = config['mlp_layers']
        # self.aggregation = config['aggregation']
        # self.num_segments_checkpoint = config.get('num_segments_checkpoint', 0)
        self.num_nodes = 100
        self.node_dim = 2
        self.voc_nodes_in = 101
        self.voc_nodes_out = 2
        self.voc_edges_in = 3
        self.voc_edges_out = 2
        self.hidden_dim = 6
        self.num_layers = 30
        self.mlp_layers = 3
        self.aggregation = "mean"
        self.num_segments_checkpoint = 5

        # Node and edge embedding layers/lookups
        self.nodes_coord_embedding = nn.Linear(self.node_dim, self.hidden_dim // 2, bias=False)
        # self.nodes_coord_embedding2 = nn.Linear(self.node_dim, self.hidden_dim, bias=False)
        self.edges_values_embedding = nn.Linear(1, self.hidden_dim // 2, bias=False)
        # self.edges_values_embedding2 = nn.Linear(1, self.hidden_dim, bias=False)
        self.edges_embedding = nn.Embedding(self.voc_edges_in, self.hidden_dim // 2)
        # self.edges_embedding2 = nn.Embedding(self.voc_edges_in, self.hidden_dim)
        self.nodes_embedding = nn.Embedding(self.voc_nodes_in, self.hidden_dim // 2)
        # self.nodes_embedding2 = nn.Embedding(self.voc_nodes_in, self.hidden_dim)
        # Define GCN Layers
        gcn_layers = []
        for layer in range(self.num_layers):
            gcn_layers.append(ResidualGatedGCNLayer(self.hidden_dim, self.aggregation))
        self.gcn_layers = nn.ModuleList(gcn_layers)
        # Define MLP classifiers
        self.mlp_edges = MLP(self.hidden_dim, self.voc_edges_out, self.mlp_layers)
        # self.mlp_nodes = MLP(self.hidden_dim, self.voc_nodes_out, self.mlp_layers)

    def forward(self, x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges=None, edge_cw=None):
        """
        Args:
            x_edges: Input edge adjacency matrix (batch_size, num_nodes, num_nodes)
            x_edges_values: Input edge distance matrix (batch_size, num_nodes, num_nodes)
            x_nodes: Input nodes (batch_size, num_nodes)
            x_nodes_coord: Input node coordinates (batch_size, num_nodes, node_dim)
            y_edges: Targets for edges (batch_size, num_nodes, num_nodes)
            edge_cw: Class weights for edges loss
            # y_nodes: Targets for nodes (batch_size, num_nodes, num_nodes)
            # node_cw: Class weights for nodes loss

        Returns:
            y_pred_edges: Predictions for edges (batch_size, num_nodes, num_nodes)
            # y_pred_nodes: Predictions for nodes (batch_size, num_nodes)
            loss: Value of loss function
        """
        # Node and edge embedding
        ## Todo: fix this but gives bugs for now
        x_vals = self.nodes_coord_embedding(x_nodes_coord)  # B x V x H
        x_tags = self.nodes_embedding(x_nodes)
        x = torch.cat((x_vals, x_tags), -1)
        # x = self.nodes_embedding2(x_nodes)
        e_vals = self.edges_values_embedding(x_edges_values.unsqueeze(3))  # B x V x V x H
        e_tags = self.edges_embedding(x_edges)  # B x V x V x H
        e = torch.cat((e_vals, e_tags), -1)
        #e = self.edges_values_embedding2(x_edges_values.unsqueeze(3))
        # GCN layers
        if self.num_segments_checkpoint != 0:
            layer_functions = [lambda args: layer(*args) for layer in self.gcn_layers]
            x, e = torch.utils.checkpoint.checkpoint_sequential(layer_functions, self.num_segments_checkpoint, (x, e))
        else:
            for layer in range(self.num_layers):
                # B x V x H, B x V x V x H
                x, e = self.gcn_layers[layer](x, e)
        # MLP classifier
        y_pred_edges = self.mlp_edges(e)  # B x V x V x voc_edges_out
        # y_pred_nodes = self.mlp_nodes(x)  # B x V x voc_nodes_out

        #loss = loss_edges(y_pred_edges, y_edges, edge_cw)
        # Edge loss
        y = F.log_softmax(y_pred_edges, dim=3)  # B x V x V x voc_edges
        
        y = torch.sigmoid(y)
        y = torch.relu(y)
        
        # For some reason we must make things contiguous to prevent errors during backward
        y_perm = y.permute(0, 3, 1, 2).contiguous()  # B x voc_edges x V x V
        # y_perm = y.permute(0, 3, 1, 2).contiguous()
        # if edge_cw == None:
        #     edge_cw = [1 for i in range(101)]
        # if type(edge_cw) != torch.Tensor:
        #     edge_labels = y_edges.cpu().numpy().flatten()
        #     edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels).tolist()
        # print(edge_cw)
        if y_edges is not None:
            # Compute loss
            # print("forwarding1.1")
            if edge_cw != None:
                edge_cw = torch.Tensor(edge_cw)  # Convert to tensors
                # print("forwarding1.1.1")
                edge_cw = edge_cw.type(self.dtypeFloat)
            # print("forwarding1.2")
                loss = nn.NLLLoss(edge_cw)
            else:
                loss = nn.NLLLoss()
            # print("forwarding1.3")
            # print(y_perm)
            # print(y_edges)
            loss = loss(y_perm, y_edges)
            # print("forwarding1.4")
        else:
            # print("forwarding2.1")
            loss = None
            # print("forwarding2.2")
        
        # return y_pred_edges.permute(0, 3, 1, 2)[0]
        return y, loss


In [13]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in datatorch:
        # data = data.to(device)
        x_edges, x_edges_values, x_nodes, x_nodes_coord, _,  y_edges = data
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output, loss = model(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges)
        # print(output, data.y)
        # print(output)
        # loss = F.l1_loss(output, y_edges)
        loss.backward()
        loss_all += loss.item()
        optimizer.step()
        test_size = 101*101*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    test_size = 101*101*1 #extract batch size
    for data in loader:
        # data = data.to(device)
        x_edges, x_edges_values, x_nodes, x_nodes_coord, _,  y_edges = data
        output, _ = model(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges)
        pred = torch.tensor([[(1 if output[0][i][j][1] > output[0][i][j][0] else 0) for j in range(101)] for i in range(101)], dtype=torch.double)
        correct += pred.eq(y_edges).sum().item()
    return correct / test_size

In [14]:
from copy import deepcopy

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = ResidualGatedGCNModelVRP().to(device)
model = ResidualGatedGCNModelVRP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 11):
    loss = train(epoch)
    train_acc = test(datatorch)
    test_acc = test(torchtest)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: -0.0494, Train Acc: 1019.8394, Test Acc: 8.9319
Epoch: 002, Loss: -0.0500, Train Acc: 1019.8394, Test Acc: 8.9319
Epoch: 003, Loss: -0.0500, Train Acc: 1019.8394, Test Acc: 8.9319
Epoch: 004, Loss: -0.0500, Train Acc: 1019.8394, Test Acc: 8.9319


KeyboardInterrupt: 

In [ ]:
import pickle

In [ ]:
with open('torch_convnet.pickle', 'wb') as handle:
    pickle.dump(model_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     model_dict_restored = pickle.load(handle)

In [ ]:
model.eval()
data_show = datatorch[0]
x_edges, x_edges_values, x_nodes, x_nodes_coord, _,  y_edges = data_show
output, _ = model(x_edges, x_edges_values, x_nodes, x_nodes_coord, y_edges)
pred = torch.tensor([[(1 if exp(output[0][i][j][1])/(exp(output[0][i][j][1]) + exp(output[0][i][j][0])) > 0.9 and output[0][i][j][1] > 0 else 0) for j in range(101)] for i in range(101)], dtype=torch.double)
y = y_edges[0]
diff = 0
for j in range(len(pred)):
    a_list = [i.tolist() for i in pred[j]]
    diff += sum([(y[j][i].tolist() - a_list[i])**2 for i in range(len(a_list))])
    print([int(i) for i in pred[j]], sum(a_list))
print(diff)
print(sum([y[j][i].tolist() for i in range(101) for j in range(101)]))
# for i in range(101):
#     for j in range(101):
#         print(output[0][i][j].tolist())
edge_labels = y_edges.cpu().numpy().flatten()
edge_cw = compute_class_weight("balanced", classes=np.unique(edge_labels), y=edge_labels).tolist()
# print(edge_cw)
# print(y)